In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("🔍 DATA EXPLORATION - Understanding Your Indian Uber Dataset")
print("="*60)

# Step 1: Load and Basic Info
try:
    df = pd.read_csv('uber_rides_india.csv')
    print("✅ Data loaded successfully!")
except FileNotFoundError:
    print("❌ Please run the data generator script first!")
    exit()

print(f"\n📊 DATASET OVERVIEW:")
print("="*25)
print(f"Total rides: {len(df):,}")
print(f"Columns: {len(df.columns)}")
print(f"Date range: Simulated data")
print(f"Cities covered: {len(df['city'].unique())}")

# Step 2: Dataset Structure
print(f"\n📋 DATASET STRUCTURE:")
print("="*25)
print(df.info())

print(f"\n📏 DATASET SHAPE:")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

# Step 3: First Look at Data
print(f"\n👀 FIRST 5 RIDES:")
print("="*20)
print(df.head())

print(f"\n👀 LAST 5 RIDES:")
print("="*20)
print(df.tail())

# Step 4: Missing Values Check
print(f"\n❓ MISSING VALUES CHECK:")
print("="*30)
missing_data = df.isnull().sum()
if missing_data.sum() == 0:
    print("✅ No missing values found!")
else:
    print("⚠️ Missing values found:")
    print(missing_data[missing_data > 0])

# Step 5: Data Types
print(f"\n🏷️ DATA TYPES:")
print("="*20)
for col in df.columns:
    dtype = df[col].dtype
    unique_vals = df[col].nunique()
    print(f"{col:20} | {str(dtype):10} | {unique_vals:3} unique values")

# Step 6: Numerical Variables Summary
print(f"\n📊 NUMERICAL VARIABLES SUMMARY:")
print("="*40)
numerical_cols = ['hour', 'distance_km', 'base_fare', 'surge_multiplier', 'final_fare', 'wait_time_mins']
print(df[numerical_cols].describe().round(2))

# Step 7: Categorical Variables
print(f"\n📝 CATEGORICAL VARIABLES BREAKDOWN:")
print("="*45)

categorical_cols = ['city', 'day_of_week', 'weather', 'special_event', 'customer_segment', 'ride_completed']

for col in categorical_cols:
    print(f"\n🏷️ {col.upper()}:")
    value_counts = df[col].value_counts()
    percentages = (df[col].value_counts(normalize=True) * 100).round(1)
    
    for value, count in value_counts.items():
        pct = percentages[value]
        print(f"  {str(value):15} | {count:4} rides ({pct:4.1f}%)")

# Step 8: Key Business Metrics
print(f"\n💼 KEY BUSINESS METRICS:")
print("="*35)

# Overall metrics
total_rides = len(df)
completed_rides = df['ride_completed'].sum()
completion_rate = df['ride_completed'].mean()
avg_surge = df['surge_multiplier'].mean()
avg_fare = df['final_fare'].mean()
total_revenue = df[df['ride_completed']]['final_fare'].sum()

print(f"Total Ride Requests:     {total_rides:,}")
print(f"Completed Rides:         {completed_rides:,}")
print(f"Completion Rate:         {completion_rate:.1%}")
print(f"Average Surge:           {avg_surge:.2f}x")
print(f"Average Fare:            ₹{avg_fare:.2f}")
print(f"Total Revenue:           ₹{total_revenue:,.2f}")
print(f"Revenue per Request:     ₹{total_revenue/total_rides:.2f}")

# Step 9: Detect Outliers
print(f"\n🚨 OUTLIER DETECTION:")
print("="*25)

def detect_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

outlier_cols = ['base_fare', 'surge_multiplier', 'final_fare', 'distance_km', 'wait_time_mins']

for col in outlier_cols:
    outlier_count, lower, upper = detect_outliers(col)
    outlier_pct = (outlier_count / len(df)) * 100
    print(f"{col:15} | {outlier_count:3} outliers ({outlier_pct:4.1f}%) | Range: {lower:.1f} - {upper:.1f}")

# Step 10: Data Quality Issues
print(f"\n⚠️ DATA QUALITY CHECK:")
print("="*30)

# Check for impossible values
quality_issues = []

# Surge multiplier should be >= 1.0
if (df['surge_multiplier'] < 1.0).any():
    quality_issues.append("Surge multiplier < 1.0 found")

# Final fare should be >= base fare  
if (df['final_fare'] < df['base_fare']).any():
    quality_issues.append("Final fare < base fare found")

# Wait time should be positive
if (df['wait_time_mins'] <= 0).any():
    quality_issues.append("Negative or zero wait time found")

# Distance should be positive
if (df['distance_km'] <= 0).any():
    quality_issues.append("Negative or zero distance found")

if quality_issues:
    for issue in quality_issues:
        print(f"❌ {issue}")
else:
    print("✅ No major data quality issues found!")

# Step 11: Interesting Patterns to Explore
print(f"\n🔍 INTERESTING PATTERNS TO EXPLORE:")
print("="*45)

# High surge rides
high_surge = df[df['surge_multiplier'] >= 2.5]
print(f"High surge rides (≥2.5x):   {len(high_surge)} ({len(high_surge)/len(df)*100:.1f}%)")

# Rain impact
rain_rides = df[df['weather'].isin(['Light_Rain', 'Heavy_Rain'])]
print(f"Rain weather rides:         {len(rain_rides)} ({len(rain_rides)/len(df)*100:.1f}%)")

# Festival rides
festival_rides = df[df['special_event'] != 'Regular']
print(f"Festival/Event rides:       {len(festival_rides)} ({len(festival_rides)/len(df)*100:.1f}%)")

# Premium customers
premium_rides = df[df['customer_segment'] == 'Premium']
print(f"Premium customer rides:     {len(premium_rides)} ({len(premium_rides)/len(df)*100:.1f}%)")

# Late night rides (after 10 PM)
late_night = df[df['hour'] >= 22]
print(f"Late night rides (≥10 PM):  {len(late_night)} ({len(late_night)/len(df)*100:.1f}%)")

# Step 12: Quick Correlation Check
print(f"\n🔗 CORRELATION BETWEEN KEY VARIABLES:")
print("="*45)

corr_vars = ['surge_multiplier', 'base_fare', 'final_fare', 'distance_km', 'wait_time_mins', 'hour']
correlation_matrix = df[corr_vars].corr().round(2)

# Print correlation with surge_multiplier
surge_corr = correlation_matrix['surge_multiplier'].sort_values(ascending=False)
print("Correlation with Surge Multiplier:")
for var, corr in surge_corr.items():
    if var != 'surge_multiplier':
        if abs(corr) > 0.1:
            print(f"  {var:15} | {corr:+.2f} {'(Strong)' if abs(corr) > 0.5 else '(Moderate)' if abs(corr) > 0.3 else ''}")

# Step 13: Data Exploration Summary
print(f"\n📋 DATA EXPLORATION SUMMARY:")
print("="*40)
print("✅ Dataset is clean and ready for analysis")
print(f"✅ {len(df):,} rides across {df['city'].nunique()} Indian cities")
print(f"✅ No missing values or major quality issues")
print(f"✅ {df['ride_completed'].mean():.1%} overall completion rate")
print(f"✅ Average surge: {df['surge_multiplier'].mean():.2f}x")

print(f"\n🎯 READY FOR NEXT STEPS:")
print("="*30)
print("1. 📊 Data Visualization (charts and graphs)")
print("2. 🔍 Deep-dive Analysis (surge patterns)")
print("3. 🏙️ City Comparison (Mumbai vs others)")
print("4. 🌧️ Weather Impact Analysis")
print("5. 👥 Customer Segment Behavior")
print("6. 💰 Revenue Optimization")

print(f"\n💡 QUESTIONS TO EXPLORE:")
print("="*30)
print("• Which city has the highest completion rates?")
print("• How does weather affect pricing acceptance?")
print("• What's the optimal surge for each customer segment?")
print("• When are the true peak demand hours?")
print("• Do festivals really reduce price sensitivity?")

print(f"\n🚀 Type 'ready' when you want to move to visualization!")

🔍 DATA EXPLORATION - Understanding Your Indian Uber Dataset
✅ Data loaded successfully!

📊 DATASET OVERVIEW:
Total rides: 1,500
Columns: 13
Date range: Simulated data
Cities covered: 8

📋 DATASET STRUCTURE:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ride_id           1500 non-null   object 
 1   city              1500 non-null   object 
 2   hour              1500 non-null   int64  
 3   day_of_week       1500 non-null   int64  
 4   weather           1500 non-null   object 
 5   special_event     1500 non-null   object 
 6   distance_km       1500 non-null   float64
 7   base_fare         1500 non-null   float64
 8   surge_multiplier  1500 non-null   float64
 9   final_fare        1500 non-null   float64
 10  ride_completed    1500 non-null   bool   
 11  wait_time_mins    1500 non-null   int64  
 12  customer_segment  1500 